In [2]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#from scipy.stats import norm
#from scipy.stats import linregress
#import statsmodels.formula.api as smf
import csv
import scipy.optimize as opt
import pandas_datareader as pdr
from datetime import datetime

In [3]:
# function to read in data from a csv
def readPricesCSV(file):
    df = pd.read_csv(file)
    dfPrices = df["Adj Close"]
    return dfPrices

In [4]:
# function to read in stock prices from yahoo finance
def readPricesYF(start_date, end_date, tickers):
    stockPrices = pdr.get_data_yahoo(tickers, start=start_date, end=end_date)   # read in the data
    stockPrices = stockPrices.filter(like='Adj Close')   # tickers are columns and data values are adjusted closing prices with the date as the index
    stockPrices.columns = tickers   # change column names to be their tickers
    stockPrices = stockPrices.stack().swaplevel().sort_index().reset_index()    # make the data tall and clean it up so it is easier to perform analysis on
    stockPrices.columns = ['Firm','Date','Adj Close']     # rename the columns
#    stockPrices['Return'] = stockPrices.groupby('Firm')['Adj Close'].pct_change()    # get the daily returns for each ticker
    return stockPrices


In [5]:
# the objective function
def obj(weights):
    
    sum_return = np.dot(weights,means)     #get the return measure given by the weights and mean
    num_stocks = len(means)
    
    #get the risk measure given by the weights
    risk_measure = np.matmul(np.matmul(np.transpose(np.array(weights)), cov_matr), np.array(weights))
#    print("risk measure: ", risk_measure)
    
    #return the objective function value
    return -(sum_return - (risk_level * risk_measure))


### To Change:
**The following block of code is the only one in this file that should be changed.**
- Add more stocks by including them in the list of tickers
- Change the starting and ending dates

**If there are any changes made, rerun the entire code after making the changes**

**Nothing else should be changed in this file**

In [57]:
#choose which stocks to include in the optimization of the portfolio (list the tickers)
tickers = ['MSFT', 'AAPL', 'AMZN', 'NFLX', 'DIS', 'TSLA']
tickers = sorted(tickers)

#set the start and end dates- choose last six months in this case
start_date = datetime(2020, 8, 25) #year, month, day
end_date = datetime(2021,2,25) #year, month, day

stockPrices = readPricesYF(start_date, end_date, tickers)    # use the function defined above to get data from Yahoo Finance
#stockPrices['ret'] = stockPrices['Adj Close'].pct_change()
stockPrices

,Firm,Date,Adj Close
0,AAPL,2020-08-25,124.424088
1,AAPL,2020-08-26,126.116135
2,AAPL,2020-08-27,124.608498
3,AAPL,2020-08-28,124.406647
4,AAPL,2020-08-31,128.625549
...,...,...,...
757,TSLA,2021-02-19,781.299988
758,TSLA,2021-02-22,714.500000
759,TSLA,2021-02-23,698.840027
760,TSLA,2021-02-24,742.020020


In [58]:
#stockPrices

In [59]:
# reformat the data
stock_prices_format = stockPrices.set_index(['Date', 'Firm']).unstack()   # change format so that date and each company are columns
stock_prices_format.reset_index()
stock_prices_format

Adj Close                                                   \
Firm              AAPL         AMZN         DIS        MSFT        NFLX   
Date                                                                      
2020-08-25  124.424088  3346.489990  129.789993  215.408630  490.579987   
2020-08-26  126.116135  3441.850098  132.179993  220.065674  547.530029   
2020-08-27  124.608498  3400.000000  133.729996  225.469070  526.270020   
2020-08-28  124.406647  3401.800049  135.539993  227.787643  523.890015   
2020-08-31  128.625549  3450.959961  131.869995  224.424194  529.559998   
...                ...          ...         ...         ...         ...   
2021-02-19  129.869995  3249.899902  183.649994  240.970001  540.219971   
2021-02-22  126.000000  3180.739990  191.759995  234.509995  533.780029   
2021-02-23  125.860001  3194.500000  197.089996  233.270004  546.150024   
2021-02-24  125.349998  3159.530029  197.509995  234.550003  553.409973   
2021-02-25  120.989998  3057.159912  190.979996  228.990005  546.700012   

                        
Firm              TSLA  
Date                    
2020-08-25  404.667999  
2020-08-26  430.634003  
2020-08-27  447.750000  
2020-08-28  442.679993  
2020-08-31  498.320007  
...                ...  
2021-02-19  781.299988  
2021-02-22  714.500000  
2021-02-23  698.840027  
2021-02-24  742.020020  
2021-02-25  682.219971  

[127 rows x 6 columns]

In [60]:
# get the different number of stocks and the number of stock prices 
num_stocks = len(tickers)   # the number of stocks
prices_per_stock = stock_prices_format.shape[0]   # the number of stock prices

In [61]:
# get the covariance matrix
cov_matr = np.cov(np.array(stock_prices_format.iloc[:,range(0,num_stocks)].T))   # get the columns and transpose it so it is in right format, then turn it into covariance matrix
cov_matr

array([[   78.14430733,   558.11492864,   170.96505794,    84.26230921,
          180.56013459,  1312.71030178],
       [  558.11492864, 12168.77726018,   378.16645303,   810.26625573,
         2169.09772824,  3904.51569958],
       [  170.96505794,   378.16645303,   582.86500156,   198.7905976 ,
          352.71609083,  3815.26375622],
       [   84.26230921,   810.26625573,   198.7905976 ,   136.02137507,
          251.58777063,  1426.33876124],
       [  180.56013459,  2169.09772824,   352.71609083,   251.58777063,
          764.99314083,  2771.11535027],
       [ 1312.71030178,  3904.51569958,  3815.26375622,  1426.33876124,
         2771.11535027, 30260.84334216]])

In [62]:
# get the mean price for each stock

means = np.array(stock_prices_format.mean().to_list())    # get means for each column, convert to a list, convert to a numpy array
means

array([ 122.99478972, 3207.85959223,  151.33307059,  218.25363448,
        514.73921264,  581.37560579])

In [63]:
# set the risk level
risk_level = 1   # risk loving < 0; risk neutral = 0; risk averse > 0

In [64]:
# set an intial value for the weights
weights = np.array([1/num_stocks]*num_stocks) # set an initial value for the weights, which is an even composition

### At this point, we have the following information:
 - **"tickers"** is a list of the tickers, and it is in alphabetical order
 - **"num_stocks"** is the number of different stocks
 - **"prices_per_stock"** is the different number of observations/the different number of stock prices obtained

For the Optimization:
 - **"cov_matr"** is the covariance matrix
 - **"means"** is a numpy array that consists of the means of each stock price during the time frame **(in alphabetical order of the tickers)**
 - **"risk_level"** is the risk level
 - **"weights"** is a numpy array that consists of the initial value for the weights, which is just an even composition and will be changed later **(in alphabetical order of the tickers)**

In [115]:
# perform the optimization!
lin_constr = opt.LinearConstraint([1]*num_stocks, [1], [1])    # sum of all values are greater than or equal to 1 and less than or equal to 1, so the sum has to be equal to 1
bounds = opt.Bounds([0]*num_stocks, [0.5]*num_stocks)    # each portfolio weight is greater than 0 and less than 0.5
result = opt.minimize(obj, x0=[1/num_stocks]*num_stocks, method="trust-constr", constraints = lin_constr, bounds=bounds)    # actually perform the optimization


optimal_weights = pd.DataFrame({'stock': tickers, 'weights': result.x.tolist()})
optimal_weights
#sum(optimal_weights['weights'])
np.dot(optimal_weights["weights"],means) - (np.matmul(np.matmul(np.transpose(np.array(optimal_weights["weights"])), cov_matr), np.array(optimal_weights["weights"])))

print(result.fun) 
#print(result.x) 

-154.3491083785433


Next steps:
- Tweak the model to use the predicted prices instead of historical prices
- Test to see how well the portfolio works
    - Active share
        - take proportion from an existing index and tweak weights
    - Compare to performance of S&P 500 or something like that
    - Can create a graph to visualize the data
    - Calculate the information ratio
- Figure out how to add a Conditional Value at Risk measure
- Explore Nested Clustered Optimization
- Could incorporate elements from stochastic calculus about estimating stock prices

In [66]:
#!pip install PyPortfolioOpt #need to install C++ first by doing (xcode-select --install) in terminal 

#LINK to article: https://towardsdatascience.com/automating-portfolio-optimization-using-python-9f344b9380b9#

In [80]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.cla import CLA
from pypfopt import plotting
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_datareader as web
from matplotlib.ticker import FuncFormatter

In [81]:
nullin_df = pd.DataFrame(stock_prices_format)
print(nullin_df.isnull().sum()) #no null values (good!)
 

           Firm
Adj Close  AAPL    0
           AMZN    0
           DIS     0
           MSFT    0
           NFLX    0
           TSLA    0
dtype: int64


In [82]:
#Annualized Return
mu = expected_returns.mean_historical_return(stock_prices_format)
#Sample Variance of Portfolio
Sigma = risk_models.sample_cov(stock_prices_format)

In [83]:
#print(mu,Sigma)
Sigma
mu


           Firm
Adj Close  AAPL   -0.054438
           AMZN   -0.165441
           DIS     1.165177
           MSFT    0.130074
           NFLX    0.241877
           TSLA    1.842177
dtype: float64

In [84]:
# #why doesn't this match up?
# stock_firm = stockPrices.groupby(['Firm'])['ret'].mean()
# stock_firm

In [85]:
#Max Sharpe Ratio - Tangent to the EF
ef = EfficientFrontier(mu, Sigma, weight_bounds=(0,1)) #weight bounds in negative allows shorting of stocks
sharpe_pfolio=ef.max_sharpe() #May use add objective to ensure minimum zero weighting to individual stocks
sharpe_pwt=ef.clean_weights()
print(sharpe_pwt)

OrderedDict([(('Adj Close', 'AAPL'), 0.0), (('Adj Close', 'AMZN'), 0.0), (('Adj Close', 'DIS'), 0.70532), (('Adj Close', 'MSFT'), 0.0), (('Adj Close', 'NFLX'), 0.0), (('Adj Close', 'TSLA'), 0.29468)])


In [87]:
sharpe_pfolio

OrderedDict([(('Adj Close', 'AAPL'), 0.0),
             (('Adj Close', 'AMZN'), 0.0),
             (('Adj Close', 'DIS'), 0.7053186626535789),
             (('Adj Close', 'MSFT'), 0.0),
             (('Adj Close', 'NFLX'), 0.0),
             (('Adj Close', 'TSLA'), 0.294681337346421)])

In [121]:
optimal_weights

,stock,weights
0,AAPL,4.660625e-01
1,AMZN,8.554199e-02
2,DIS,2.936790e-09
3,MSFT,4.483955e-01
4,NFLX,9.698913e-09
5,TSLA,1.169189e-10


In [103]:
stock_returns = stock_prices_format.pct_change()[1:]
stock_returns

weighted_returns = weights * stock_returns
stock_returns

Adj Close                                                  
Firm            AAPL      AMZN       DIS      MSFT      NFLX      TSLA
Date                                                                  
2020-08-26  0.013599  0.028496  0.018414  0.021620  0.116087  0.064166
2020-08-27 -0.011954 -0.012159  0.011726  0.024554 -0.038829  0.039746
2020-08-28 -0.001620  0.000529  0.013535  0.010283 -0.004522 -0.011323
2020-08-31  0.033912  0.014451 -0.027077 -0.014766  0.010823  0.125689
2020-09-01  0.039833  0.013956  0.012740  0.007715  0.050967 -0.046697
...              ...       ...       ...       ...       ...       ...
2021-02-19  0.001233 -0.023535  0.003552 -0.011567 -0.014593 -0.007722
2021-02-22 -0.029799 -0.021281  0.044160 -0.026808 -0.011921 -0.085499
2021-02-23 -0.001111  0.004326  0.027795 -0.005288  0.023174 -0.021917
2021-02-24 -0.004052 -0.010947  0.002131  0.005487  0.013293  0.061788
2021-02-25 -0.034783 -0.032400 -0.033062 -0.023705 -0.012125 -0.080591

[126 rows x 6 columns]

In [106]:
#port returns are the sum of the weighted returns 

port_ret = weighted_returns.sum(axis=1)#axis =1 tels pandas to add the rows 
#port_ret

Date
2020-08-26    0.043730
2020-08-27    0.002181
2020-08-28    0.001147
2020-08-31    0.023839
2020-09-01    0.013086
                ...   
2021-02-19   -0.008772
2021-02-22   -0.021858
2021-02-23    0.004497
2021-02-24    0.011283
2021-02-25   -0.036111
Length: 126, dtype: float64

In [142]:
#Portfolio statistics (equal weighted)
mean_ret = port_ret.mean()
std_ret = port_ret.std()
sharpe = mean_ret/std_ret

#std_ret
#mean_ret



In [143]:
opt_weights = optimal_weights['weights'].to_list()
#opt_weights = np.dot(optimal_weights, stock_returns)
opt_weights
weighted_ret_opt = opt_weights * stock_returns
weighted_ret_opt

port_ret_opt = weighted_ret_opt.sum(axis=1)#axis =1 tels pandas to add the rows 
port_ret_opt

Date
2020-08-26    0.018470
2020-08-27    0.004398
2020-08-28    0.003901
2020-08-31    0.010420
2020-09-01    0.023218
                ...   
2021-02-19   -0.006625
2021-02-22   -0.027729
2021-02-23   -0.002519
2021-02-24   -0.000365
2021-02-25   -0.029612
Length: 126, dtype: float64

In [148]:
#Portfolio statistics (optimally weighted)
mean_ret_opt = port_ret_opt.mean()
std_ret_opt = port_ret_opt.std()
#std_ret
#mean_ret
sharpe_opt = mean_ret_opt/std_ret_opt

mean_diff = mean_ret_opt - mean_ret
sigma_diff = std_ret_opt - std_ret

sharpe_diff = sharpe_opt - sharpe
print(mean_diff, sigma_diff)
print(sharpe_diff) #our sharpe ratio is now lower after adjusting optimal weights BAD 

-0.0014138639139355306 0.0004035089325654037
-0.07510456991053861


In [ ]:
weighted_ret